In [1]:
import tensorflow as tf

2022-06-02 07:58:33.189915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 07:58:33.189990: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
modelo = tf.keras.models.load_model('rede1.h5')

2022-06-02 07:59:49.447843: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-02 07:59:49.468700: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-06-02 07:59:49.949167: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-02 07:59:59.144640: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-06-02 07:59:59.363657: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-06-02 07:59:59

In [5]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              1801000   
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 1)                 501       
                                                                 
Total params: 2,302,001
Trainable params: 2,302,001
Non-trainable params: 0
_________________________________________________________________


In [12]:


url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDia?diaNum=20220530"

In [14]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [15]:
df.head()

,campoX,ticker,diaNumPrevisao
0,"1.130710, 1.122980, 1.120169, 1.120169, 1.132...",ABEV3,20220530
1,"2.112299, 2.112299, 2.101604, 2.106952, 2.112...",AERI3,20220530
2,"1.698991, 1.657306, 1.678806, 1.668714, 1.673...",ALPA4,20220530
3,"1.033316, 1.010411, 1.023946, 1.028110, 1.034...",ALSO3,20220530
4,"1.395257, 1.367589, 1.375494, 1.375494, 1.391...",AMAR3,20220530


In [16]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [17]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(88, 1800)

In [18]:
result = modelo.predict(X)

In [19]:
type(result)

numpy.ndarray

In [20]:
df['resultado'] = result

In [21]:
df.head()

,campoX,ticker,diaNumPrevisao,resultado
0,"1.130710, 1.122980, 1.120169, 1.120169, 1.132...",ABEV3,20220530,0.160397
1,"2.112299, 2.112299, 2.101604, 2.106952, 2.112...",AERI3,20220530,0.239546
2,"1.698991, 1.657306, 1.678806, 1.668714, 1.673...",ALPA4,20220530,0.352048
3,"1.033316, 1.010411, 1.023946, 1.028110, 1.034...",ALSO3,20220530,0.128953
4,"1.395257, 1.367589, 1.375494, 1.375494, 1.391...",AMAR3,20220530,0.292051


In [22]:
df.drop('campoX', inplace=True, axis=1)

In [23]:
json = df.to_json()

In [24]:
print(json)

{"ticker":{"0":"ABEV3","1":"AERI3","2":"ALPA4","3":"ALSO3","4":"AMAR3","5":"ANIM3","6":"AZUL4","7":"B3SA3","8":"BBAS3","9":"BBDC4","10":"BBSE3","11":"BEEF3","12":"BKBR3","13":"BPAC11","14":"BPAN4","15":"BRAP4","16":"BRFS3","17":"BRKM5","18":"BRML3","19":"CCRO3","20":"CIEL3","21":"CMIG4","22":"COGN3","23":"CPFE3","24":"CPLE6","25":"CRFB3","26":"CSAN3","27":"CSMG3","28":"CSNA3","29":"CVCB3","30":"CYRE3","31":"ECOR3","32":"EGIE3","33":"ELET3","34":"ELET6","35":"EMBR3","36":"ENBR3","37":"ENEV3","38":"ENGI11","39":"EQTL3","40":"EZTC3","41":"FLRY3","42":"GGBR4","43":"GOAU4","44":"GOLL4","45":"HAPV3","46":"HYPE3","47":"ITUB4","48":"JBSS3","49":"JHSF3","50":"KLBN11","51":"LCAM3","52":"LIGT3","53":"LREN3","54":"LWSA3","55":"MGLU3","56":"MOVI3","57":"MRFG3","58":"MRVE3","59":"MULT3","60":"MYPK3","61":"NEOE3","62":"NTCO3","63":"PCAR3","64":"PETR4","65":"POSI3","66":"PRIO3","67":"PTBL3","68":"QUAL3","69":"RADL3","70":"RAIL3","71":"RAPT4","72":"RENT3","73":"SANB11","74":"SBSP3","75":"SLCE3","76":"S

In [27]:
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisao'
x = requests.post(url, data = json)

## Experiencias

In [ ]:
dfTicker = df.set_index('ticker')
dfTicker.head()

In [ ]:
dfTratadoX = dfTicker.apply(trataLinhaX, axis='columns')

In [ ]:
dfTratadoX.head()

In [ ]:
result = modelo.predict(dfTratadoX)